**Objectif :**  
Obtenir un dataset permettant de détecter le **front running** à l'aide du **machine learning**, tout en suivant l'activité d'une famille d'adresses.

Il serait intéressant d'ajouter des informations telles que :  
- Le score de liquidité d'un token.  
- Le prix des tokens achetés et vendus au moment de la confirmation et lors de leur entrée dans le **mempool**.

Concernant les types de swaps : trouver un moyen de distinguer les swaps automatisés et non automatisés (comme l'exemple des 71 swaps et celui des swaps agrégés).

Il est nécessaire de concentrer ce travail sur un seul bloc, car les transactions y sont classées de manière chronologique.

# Sommaire

1. Étude de block
2. Distribution des données au sein du block

Quelle table manipuler ?
- dex.trades
- dex.sandwiches
- dex.sandwiched
- transfers_ethereum.eth

# Quel type de transaction est auteur de front running ?

#### Récupération des données provenant de dune du 20/09/2024

In [29]:
# Importation des 2 données provenant de dune
import pandas as pd
list = pd.read_csv('/Users/dylan/sand_20_09_24.csv')
df = pd.read_csv('/Users/dylan/trad_20_09_24.csv')

In [30]:
# Rassemble les swaps étant auteurs de front running ou back running
list.sort_values(by='block_time', inplace=True)

Selon la table `labels.sandwich_attackers`, toutes les adresses identifiées comme réalisant des attaques de type **sandwich** sur Ethereum l'ont fait exclusivement via des **échanges décentralisés (DEX)**. Cela signifie que ces attaques exploitent la transparence et le délai d'exécution des transactions sur les DEX pour intercepter des échanges en cours et générer un profit.

### Logique d'identification d'un attaquant de sandwich
Pour identifier une adresse réalisant des attaques sandwich, la logique suivante est appliquée :

1. Deux transactions distinctes, \( t1 \) et \( t2 \), dans le même bloc initiées par la même adresse.
2. Les deux transactions sont effectuées sur le même projet.
3. Le token acheté dans \( t1 \) est vendu dans \( t2 \).
4. Le token vendu dans \( t1 \) est acheté dans \( t2 \).
5. La quantité achetée dans \( t1 \) est égale à celle vendue dans \( t2 \).
6. La quantité vendue dans \( t1 \) est inférieure à celle achetée dans \( t2 \) (ce cas est exclu pour éviter les faux positifs).
7. L'index de \( t1 \) est supérieur ou égal à \( t2 + 2 \), ou inversement (cela permet d'inclure les attaquants qui ne regroupent pas leurs transactions via Flashbots).
8. Exclusion de l'adresse du routeur Uniswap v2, souvent inclus à tort.

Cette méthode permet d'identifier à la fois les attaques **buy-first** (achat en premier) et, moins courantes, les attaques **sell-first** (vente en premier), tout en incluant les attaques sandwich qui ont pu être déficitaires.

Ici, il pourrait-être utile de récupérer de l'information ! De plus, on traitera les attaques `buy-first`

In [74]:
# Une partie des données d'échanges effectuées le 20/09/2024
df.sort_values(by='block_time', inplace=True)

In [32]:
df.dtypes

blockchain                   object
project                      object
version                      object
block_month                  object
block_date                   object
block_time                   object
block_number                float64
token_bought_symbol          object
token_sold_symbol            object
token_pair                   object
token_bought_amount          object
token_sold_amount            object
token_bought_amount_raw      object
token_sold_amount_raw        object
amount_usd                   object
token_bought_address         object
token_sold_address           object
taker                        object
maker                        object
project_contract_address     object
tx_hash                      object
tx_from                      object
tx_to                        object
evt_index                     int64
dtype: object

In [33]:
list.dtypes

blockchain                   object
project                      object
version                      object
block_time                   object
block_month                  object
block_number                float64
token_sold_address           object
token_bought_address         object
token_sold_symbol            object
token_bought_symbol          object
maker                        object
taker                        object
tx_hash                      object
tx_from                      object
tx_to                        object
project_contract_address     object
token_pair                   object
tx_index                      int64
token_sold_amount_raw        object
token_bought_amount_raw      object
token_sold_amount            object
token_bought_amount          object
amount_usd                   object
evt_index                     int64
dtype: object

# Analyse de list

Voyons combien de transactions sont stockées ?

In [36]:
len(list['tx_hash'].unique())

7136

Il y'a 7136 hash uniques de transactions !

## Comment savoir si un swap a été automatisée ?

Pour déterminer si un swap a été automatisé sur une plateforme de finance décentralisée (DeFi), plusieurs éléments peuvent être pris en compte :

1. **Identifiant de la transaction** : Les transactions automatisées sont souvent liées à des contrats intelligents. Si une transaction a été initiée par un contrat plutôt que par une adresse d'utilisateur, il est probable qu'elle soit automatisée. Vous pouvez consulter le **hash de la transaction** sur des explorateurs de blockchain comme Etherscan pour identifier l'origine.

2. **Gas Price et Timing** : Les transactions automatisées, comme celles effectuées par des bots de trading, peuvent avoir des prix de gas très compétitifs et être chronométrées pour profiter de mouvements de marché spécifiques. Une transaction réalisée à un moment précis avec un gas price élevé peut indiquer une activité automatisée.

3. **Caractéristiques des swaps** : Les swaps automatisés, notamment ceux réalisés par des **robots de trading** ou des **protocoles d'arbitrage**, suivent souvent des schémas répétitifs. Par exemple, des transactions fréquentes entre les mêmes paires de tokens en très peu de temps peuvent indiquer une automatisation.

4. **Interactions avec des DEX spécifiques** : Certaines plateformes, comme Uniswap ou Sushiswap, permettent aux utilisateurs de programmer des transactions. Si une transaction est liée à une telle fonctionnalité, elle pourrait être automatisée.

5. **Historique des transactions** : En examinant l'historique des transactions d'une adresse, vous pouvez repérer des modèles d'activité. Si une adresse montre des comportements consistants avec des échanges fréquents et rapides, cela peut indiquer l'utilisation de scripts ou de bots.

Pour plus d'informations sur la manière d'analyser les transactions sur des plateformes de blockchain, consultez des ressources comme [Etherscan](https://etherscan.io) ou des articles spécialisés sur le trading algorithmique et les bots de trading dans la DeFi.

## Combien et quels types d'actions sont associés à un même identifiant de transaction ?

In [127]:
# Compter le nombre d'observations par 'tx_hash'
observations_per_hash = list['tx_hash'].value_counts()

In [129]:
observations_per_hash

tx_hash
0x2d1a53e1de5a643fe8e707c63bbf2d0036774bbeae03f391581901652d6f79cc    71
0x30ea32f4555f06b7575d03733155828d5d63b559b250866bea4660fee7be436e    42
0x342f6d331a22b9ce1466bb2a1fb9103baf23587ce7482fe43fb0fcc83b69d38c    14
0x36037a7180cffb16203cea19a794192a80ba4546cda2c278b8a876df710b2318    13
0x71d1c63027914f26dbce941e6e92aa16ee4d78e924f8d8bd10deace1b4f29c54     7
                                                                      ..
0x087f9c312ec87722d2b8415c3d8ba8000040e993c7741f145a4b333386eeeb10     1
0xae4d36f58cb32da32a61a99e72f69898ebdef6df356150a5aaa480ac2496f8d6     1
0x2f2c1dcc9cb2dbbe41fc689237500228051e04058c6a8cd2815d23a435dafaab     1
0x63698553494ae7659b31a1fa9ae9fbb52060be94d178234d3090700d21f390aa     1
0x08c44eb6ede2a5da4667648e58d7fc40b0a31e4927b5bfa670bdb24ffdb9a0a6     1
Name: count, Length: 7136, dtype: int64

## Étude d'une transaction ayant plus de 71 actions de transactions

In [123]:
# Sélectionner les colonnes dont le nom contient 'token', 'amount_usd', 'tx_index' ou 'index'
selected_columns = list.columns[list.columns.str.contains('token|amount_usd|tx_index|index')]

# Filtrer les lignes où 'tx_hash' correspond au hash spécifié
filtered_rows = list[list['tx_hash'] == '0x2d1a53e1de5a643fe8e707c63bbf2d0036774bbeae03f391581901652d6f79cc']

# Trier les lignes filtrées par 'evt_index' dans l'ordre croissant
filtered_rows_sorted = filtered_rows.sort_values(by='evt_index')

# Obtenir les 50 dernières lignes triées avec les colonnes sélectionnées
filtered_rows_sorted[selected_columns].tail(50)

,token_sold_address,token_bought_address,token_sold_symbol,token_bought_symbol,token_pair,tx_index,token_sold_amount_raw,token_bought_amount_raw,token_sold_amount,token_bought_amount,amount_usd,evt_index
3162,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,90218195159342087328357,125939868460195931,90218.19515934208,0.12593986846019595,322.120179756697,682
3263,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,12500000000000000000000,17233212033894767,12499.999999999998,0.017233212033894767,44.07790341545366,688
6700,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,90218195159342087328357,122859343073866650,90218.19515934208,0.12285934307386666,314.24102756032096,694
5737,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,12500000000000000000000,16814263284016536,12499.999999999998,0.016814263284016535,43.006345629427614,700
3855,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,90218195159342087328357,119890531448043351,90218.19515934208,0.11989053144804335,306.6476090006039,706
546,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,12500000000000000000000,16410415479051512,12499.999999999998,0.01641041547905151,41.97341198323442,712
4919,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,90218195159342087328357,117028095181197006,90218.19515934208,0.11702809518119701,299.32626988780305,718
3725,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,90218195159342087328357,114267011029849291,90218.19515934208,0.11426701102984929,292.26416212137644,730
9600,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,90218195159342087328357,111602548840299960,90218.19515934208,0.11160254884029996,285.44918724530044,742
8060,0xa90267024007e60484d080d7951fb63830603e63,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,FRAK,WETH,FRAK-WETH,2,12500000000000000000000,15282512331130773,12499.999999999998,0.015282512331130772,39.08854026470311,748


Il est possible que certains **hashs** se répètent, car un identifiant de transaction peut être associé à plusieurs actions (transfert, échange, etc.). Il sera donc nécessaire de développer un algorithme permettant d'identifier précisément chaque hash susceptible de faire du front running, du back running ou un échange simple. Cela facilitera la **classification binaire** de ces transactions.

Grâce à l'identifiant `0x2d1a53e1de5a643fe8e707c63bbf2d0036774bbeae03f391581901652d6f79cc`, nous avons observé un swap global séquencé en 71 swaps, où le token vendu est `FRAK` et le token acheté est `WETH`.

# Quel type de swap traitons-nous ?

En parallèle de nos analyses, nous utilisons Etherscan pour obtenir des informations supplémentaires sur chaque identification de transaction.

Après avoir étudié cette situation, on suppose qu'une transaction liée à plusieurs actions consiste à effectuer un swap de manière séquencée, probablement pour ne pas attirer l'attention. Vérifions si cette hypothèse est vraie pour chaque identifiant tel qu'ils aient une `value_counts`> 1 dans la liste `observations_per_hash`

In [106]:
# Conserver les observations ayant un count strictement supérieur à 1
observations_with_multiple_counts = observations_per_hash[observations_per_hash > 1]

Comment définir `observations_with_multiple_counts`? **`observations_with_multiple_counts`** est une variable qui contient uniquement les transactions (identifiées par un hash) ayant un nombre d'observations strictement supérieur à 1. C'est un sous-ensemble de **`observations_per_hash`**, qui filtre les données pour ne garder que les transactions apparaissant plus d'une fois.

In [174]:
observations_with_multiple_counts

tx_hash
0x2d1a53e1de5a643fe8e707c63bbf2d0036774bbeae03f391581901652d6f79cc    71
0x30ea32f4555f06b7575d03733155828d5d63b559b250866bea4660fee7be436e    42
0x342f6d331a22b9ce1466bb2a1fb9103baf23587ce7482fe43fb0fcc83b69d38c    14
0x36037a7180cffb16203cea19a794192a80ba4546cda2c278b8a876df710b2318    13
0x71d1c63027914f26dbce941e6e92aa16ee4d78e924f8d8bd10deace1b4f29c54     7
                                                                      ..
0xdac87c13321c86d4a8fa89f7fed2ed60819c6042167763073813f2d7e4e48c49     2
0x48332239698aef78b78d627ac793e8d470b141ea827fe28a4060fbdf3a28d2c2     2
0x8bec9653e6e63b0c08e75a1f43661b211892f13a80f54cf4294e840a6e73e47a     2
0x1e8b64a8640182b2188c5c16aaab5a63e6b48bde4a9d35ba047987773a2c4b94     2
0xc05879b577114b9c130511461ce70b550c401865892595674df028df2c60f425     2
Name: count, Length: 2013, dtype: int64

In [120]:
# Sélectionner les colonnes dont le nom contient 'token', 'amount_usd', 'tx_index' ou 'index'
selected_columns = list.columns[list.columns.str.contains('token|amount_usd|tx_index|index')]

# Filtrer les lignes où 'tx_hash' correspond au hash spécifié
filtered_rows = list[list['tx_hash'] == '0xc05879b577114b9c130511461ce70b550c401865892595674df028df2c60f425']

# Trier les lignes filtrées par 'evt_index' dans l'ordre croissant
filtered_rows_sorted = filtered_rows.sort_values(by='evt_index')

# Obtenir les 50 dernières lignes triées avec les colonnes sélectionnées
filtered_rows_sorted[selected_columns].tail(50)

,token_sold_address,token_bought_address,token_sold_symbol,token_bought_symbol,token_pair,tx_index,token_sold_amount_raw,token_bought_amount_raw,token_sold_amount,token_bought_amount,amount_usd,evt_index
4060,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x943af2ece93118b973c95c2f698ee9d15002e604,WETH,DUEL,DUEL-WETH,4,150669356672483328,126737237139813979652096,0.15066935667248332,126737.23713981398,377.2308683008965,61
4220,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x8baf5d75cae25c7df6d1e0d26c52d19ee848301a,WETH,CATALORIAN,CATALORIAN-WETH,4,24587545325076480,19002403516239474244978,0.02458754532507648,19002.403516239472,61.559837230393974,70


Voici les définition de swaps agrégées et de swap multiples !

1. **Swap agrégé** : Un swap agrégé fait référence à une transaction qui combine plusieurs échanges ou transactions individuelles en une seule opération. Cela permet de regrouper différentes opérations dans une seule transaction pour réduire les frais de réseau ou pour simplifier l'exécution. Par exemple, plusieurs échanges de jetons différents peuvent être exécutés simultanément dans une seule transaction.

2. **Swap multiple** : Un swap multiple désigne une série de swaps effectués de manière successive ou en parallèle, mais qui sont toujours distincts les uns des autres. Cela signifie qu'il y a plusieurs transactions ou échanges individuels, mais qui sont traités de manière séparée au sein de la même entité ou contrat intelligent.

### Distinction :
- Le **swap agrégé** regroupe plusieurs opérations en une seule, alors que le **swap multiple** implique plusieurs opérations distinctes, même si elles se déroulent dans une courte période.

Construisons un algorithme qui, pour chaque identifiant de transaction ayant un nombre de transactions > 2, indique les types de tokens différents vendus et les types de tokens différents achetés.

In [178]:
import pandas as pd

# Supposons que 'list' est ton DataFrame et 'observations_with_multiple_counts' est une série avec des hash en index.

# Filtrer le DataFrame pour ne garder que les tx_hash présents dans 'observations_with_multiple_counts'
filtered_transactions = list[list['tx_hash'].isin(observations_with_multiple_counts.index)]

# Grouper par tx_hash et compter les modalités différentes pour token_sold_symbol et token_bought_symbol
modalities_count = filtered_transactions.groupby('tx_hash').agg({
    'token_sold_symbol': 'nunique',  # Compte le nombre de modalités uniques pour token_sold_symbol
    'token_bought_symbol': 'nunique'  # Compte le nombre de modalités uniques pour token_bought_symbol
})

In [180]:
modalities_count

,token_sold_symbol,token_bought_symbol
tx_hash,,
0x00562d8b7a2d61c688049e32d4fc9797f80c18a812fb06e45c17a4de6486ceea,2,3
0x0073d291f1eb392b14f09abfdd3073154c1a4774e54f930aef4acb2afbcee338,2,2
0x00a1be62820b14886cc9b5b5825706533e7f3c151354f1f0c9b6c86a8023c47f,1,2
0x00b387bec9b86d6851b6b0f793ff0c441ae98ce0ad9b35608d1fb50216fe61d8,3,1
0x00c3116050c1ee718212a4b4d47b8237552cb949ce056c50a6117bdec582cc84,2,1
...,...,...
0xff5e9a87fad0ef5781ca7e19877a6f1b110631276c39553c20cbe3d9bd7ba59d,1,2
0xff702d232ec8458cfedaad0e57b296b195d866ba81a811c5341faa7933682118,2,1
0xff774bad8c1e7ffcb4517263091350f0256340ad5b2090aad8d6e4fbee441380,2,2


Il est donc nécessaire d'interpréter ces résultats et de vérifier s'il existe une relation avec le front running ou le back running

In [ ]:
import pandas as pd

# Supposons que 'list' est ton DataFrame et 'observations_with_multiple_counts' est une série avec des hash en index.

# Filtrer le DataFrame pour ne garder que les tx_hash présents dans 'observations_with_multiple_counts'
filtered_transactions = list[list['tx_hash'].isin(observations_with_multiple_counts.index)]

# Créer une nouvelle colonne avec la paire (token_sold_symbol, token_bought_symbol)
filtered_transactions['token_pair'] = list(zip(filtered_transactions['token_sold_symbol'], filtered_transactions['token_bought_symbol']))

# Récupérer les paires uniques de token_sold_symbol et token_bought_symbol
unique_token_pairs = filtered_transactions['token_pair'].drop_duplicates()

# Afficher le résultat
print(unique_token_pairs)


# Quelles différences entre ETH et WETH ?

La question que l'on peut se poser est : quelle information nous aide à déterminer si une transaction est un back running ou un front running ?

# Questions secondaires

#### Sur quels projets se produisent ces attaques ?

In [44]:
list['project'].unique()

array(['uniswap', 'verse_dex', 'sushiswap', 'pancakeswap', 'curve',
       'dodo', 'solidly', 'balancer', 'defiswap', 'shibaswap'],
      dtype=object)

#### Quelles sont les identifiants de swap en communs ?

In [46]:
elements_communs = set(list['tx_hash']) & set(df['tx_hash'])

In [47]:
len(elements_communs)

199

#### Parmi les échanges de `list`, quel est le bloc qui se répète le plus souvent ?

In [49]:
list['block_number'].value_counts()

block_number
20792892.0    114
20792715.0     20
20793938.0     15
20792240.0     12
20791207.0     11
             ... 
20792936.0      1
20792934.0      1
20792931.0      1
20792922.0      1
20795238.0      1
Name: count, Length: 3983, dtype: int64

In [50]:
# Compter la fréquence des numéros de blocs
block_most_repeated = list['block_number'].value_counts().idxmax()

# Afficher le bloc le plus répété
print("Le block le plus répété est :", block_most_repeated)

Le block le plus répété est : 20792892.0


#### Affichons tous les hash venant de dex.sandwiches

In [52]:
list['tx_hash'].unique()[:100]

array(['0xfa8586e7aef8aaf9a1261441f6bf7cabf987e2e2adfc2470bf9a16c588de66b7',
       '0xd488da5517c2ec6bf973e550adcda1ff1ee8c4e298902b490dace7a9ac9cc0fc',
       '0xeb4bcc2fd62b1501f394d63329bf8843a9e133dc1c74a1fd1bb2730866390dda',
       '0x7d21d921b70a740bd900e83e971a32e891f360eedc4c353029275b0a88e2d7c2',
       '0x38c9de48f9f4563e3c6687a21216026b17e08238825d99bd6bd5151c25d98a62',
       '0x34875dfda3485edbf6955e06f5d225352a922cba5392080accc51f80e8aacb26',
       '0xe5f66fd30a0f7262208d6aa79108366fa54e345b5937ae566b1025e6d341e1db',
       '0x0d85dc3194a1d653071f4e7fef23accd562a89747af25b9edfb332ebfa0d4e7d',
       '0x4d087b5dbff35cd7100f761420b92d362092594e385defab454a69036acace2d',
       '0x1734cd4bec2795a1de5bccec5c9254f72b36103e98f9ab0b14ee19d04bb35692',
       '0x3e1735ac3aa8a1e89e8dd820f00ceaf3586867fb1e8096045b3d89d8c2827f3e',
       '0xef69dc2595ffab9fb06d28301af40909345560a9c6ee3b4995791f58d8fc36b1',
       '0xcf9a6594f41f4df3af64bd6697e1947e349f362108b814f0748d5f0df36a9b44',

# Récupération des données provenant de Blocknative du 20/09/24

In [54]:
import pandas as pd
folder_path='/Volumes/my_passport/csv_gz_2024_09_20/csv/'

# Noms des fichiers CSV à fusionner
files = ["00.csv", "01.csv", "02.csv", "03.csv", "04.csv", "05.csv", "06.csv", "07.csv",
         "08.csv", "09.csv", "10.csv", "11.csv", "12.csv", "13.csv", "14.csv", "15.csv", 
         "16.csv", "17.csv", "18.csv", "19.csv", "20.csv", "21.csv", "22.csv", "23.csv"]

# Lecture des fichiers CSV et concaténation dans un seul DataFrame
df_list = [pd.read_csv(folder_path + file, sep='\t', low_memory=False) for file in files]
df_union_2024_09_20 = pd.concat(df_list, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/my_passport/csv_gz_2024_09_20/csv/00.csv'

In [ ]:
df_union_2024_09_20

In [ ]:
# Information essentielle des données de blocknative
df_union_2024_09_20.info()

# Voyons comment sont distribuées la sources des échanges dans le block numéro 20792892.0

In [ ]:
list[list['block_number']==20792892]['tx_hash']

In [ ]:
df[df['block_number']==20792892.0]